In [4]:
import pandas as pd
import numpy as np


In [33]:
egcos_src_data = '../data/data_clean/polisumm_final.csv'
cmos_src_data  = '../data/data_clean/polisumm_final_cmos.csv'
zs_src_data = '../data/data_clean/polisumm_eval_short_%s.csv'
fs_src_data = '../data/data_clean/polisumm_fs_test.csv'

In [15]:
src_fp    = ''
egcos_fp  = './EG COS Baseline/preds_egcos.csv'
cmos_fp   = './CMOS Baseline/preds_cmos.csv'

neural_base = '../examples/ex_summs_%s_%s.txt' # model and task type

# Supervised 

## Extractive Baselines 

In [101]:
egcos_src   = pd.read_csv(egcos_src_data)
cmos_src    = pd.read_csv(cmos_src_data)

egcos_preds = pd.read_csv(egcos_fp)
cmos_preds  = pd.read_csv(cmos_fp)

In [102]:
egcos_all = pd.concat((egcos_src, egcos_preds), axis = 1)
cmos_all  = pd.concat((cmos_src, cmos_preds), axis = 1)

In [104]:
cmos_all = cmos_all.rename({'0': 'cmos_preds'}, axis = 1)[['all_texts', 'article_url', 'title', 'left_sum', 'right_sum', 'cmos_preds']]
egcos_all = egcos_all.rename({'0': 'egcos_preds'}, axis = 1)[['all_texts', 'article_url', 'title', 'left_sum', 'right_sum', 'egcos_preds']]

In [108]:
egcos_all

,all_texts,article_url,title,left_sum,right_sum,egcos_preds
0,Mnuchin: Family of 4 could get $3K under virus...,https:/theflipside.io/archives/checks-in-the-mail,Checks In The Mail,The left supports cash payments and argues tha...,The right is generally supportive of helping c...,economy amid coronavirus outbreak | Family 4 c...
1,Analysis | What Trump's Trillion-Dollar Bailou...,https:/theflipside.io/archives/eos-regarding-t...,EOs Regarding Transgender Rights,The left is supportive of both policies.,The right is critical of both policies.,What Trump's Trillion-Dollar Bailout Gets Righ...
2,5 major Facebook stories broke over the last f...,https:/theflipside.io/archives/facebook-and-br...,Facebook and Breaking Up Big Tech,"The left is skeptical of Zuckerberg’s motives,...",The right is disturbed by the free speech impl...,I tend think thinly veiled excuse | 5 major Fa...
3,Biden maintains a 90% chance of winning.\n\nht...,https:/theflipside.io/archives/general-electio...,General Election Update,The left is optimistic about Biden’s chances.,The right is cautiously optimistic about Trump...,*wildly* uninformed guess #USElections2020 | B...
4,"Biden has a 90% chance of winning, according t...",https:/theflipside.io/archives/general-electio...,General Election Update,The left is optimistic about Biden’s chances.,The right is cautiously optimistic about Trump...,"Biden 90% chance winning, according forecast |..."
...,...,...,...,...,...,...
1194,I texted a bad diaper rash to Will's physician...,https:/theflipside.io/archives/apple-to-scan-f...,Apple to Scan for CSAM,NaN,NaN,I texted bad diaper rash Will's physician (fir...
1195,Of interest? https://t.co/H2TOBxv32t|21 cases ...,https:/theflipside.io/archives/it-was-the-best...,"It was the best of times, it was the worst of ...",NaN,NaN,We built friends (and experts | cases contesti...
1196,@rodrigo131288 @Ezequielopez096 Otro dato inte...,https:/theflipside.io/archives/arkansas-bans-t...,Arkansas Bans Transgender Treatment for Children,"The left opposes the law, arguing that the mos...","The right supports the law, arguing that few l...",@rodrigo131288 @Ezequielopez096 Otro dato inte...
1197,He's beginning to lose @CNN\n👀\nhttps://t.co/r...,https:/theflipside.io/archives/afghanistan-and...,Afghanistan and the World,NaN,NaN,He's beginning lose @CNN 👀 fun watch libs tear...


In [110]:
ext_all = egcos_all
ext_all['cmos_preds'] = cmos_all['cmos_preds']

## Abstractive Baselines 

In [36]:
def get_neural_res(model, task_type):
    
    if task_type == 'zs':
        with open(neural_base % (model, task_type + '_l'), 'r', encoding = 'utf8') as f:
            preds = f.readlines()
        return preds
            
    else:
        with open(neural_base % (model, task_type), 'r', encoding = 'utf8') as f:
            preds = f.readlines()
        return preds

In [60]:
bart_zs = get_neural_res('bart', 'zs')
bart_fs = get_neural_res('bart', 'fs')
primera_zs = get_neural_res('primera', 'zs')
primera_fs = get_neural_res('primera', 'fs')

In [61]:
zs_data = pd.read_csv(zs_src_data % 'l')[['tweet_text', 'reddit_text', 'title', 'left_sum', 'right_sum']]
fs_data = pd.read_csv(fs_src_data)[['tweet_text', 'reddit_text', 'title', 'left_sum', 'right_sum']]

In [62]:
zs_data['bart_preds'] = bart_zs
zs_data['primera_preds'] = primera_zs

In [66]:
fs_data['bart_preds'] = list(zip(bart_fs[:len(bart_fs)//2], bart_fs[len(bart_fs)//2:]))
# fs_data['primera_preds'] = list(zip(primera_fs[:len(primera_fs)//2], primera_fs[len(primera_fs)//2:]))

In [69]:
fs_data[fs_data['bart_preds'].str.slice(0, 9) == 'Joe Biden']

,tweet_text,reddit_text,title,left_sum,right_sum,bart_preds


In [72]:
fs_data['bart_preds'].str.slice(0, 9)[7]

('Joe Biden, the vice-president of the United States, is facing tests in the Middle East, and here are some of the reactions on social media.\n',
 "As the Senate prepares to vote on Brett Kavanaugh's Supreme Court nomination, here's a round-up of your reaction.\n")

# Combined FS

In [113]:
ext_all[ext_all['cmos_preds'].str.contains('babies')]['all_texts'].values

array(['@atrupar @kimguilfoyle Constructs a straw man?\nNo way pal.  The Democrats ARE murdering babies born alive!\nIt is fact!  Are you really that indoctrinated &amp; BLIND, DEAF &amp; DUMB to not see this?\nhttps://t.co/VOlQ7gKg0q|@HOPE__Messenger @HedwigGraymalk @DrJenGunter @ManInTheHoody You\'re a liar. They blocked the bill and said why: because infanticide is already illegal.\n\nIts telling that you crazy fanatics have to lie. \n\nhttps://t.co/Oynr9o7xsT|@corycolt @realDonaldTrump https://t.co/Yyl3AqzXrn|Is it now an extremist view that you shouldn\'t be able to kill a child born alive, breathing on the table, who survived an abortion attempt? https://t.co/jbXTOKCPtX|@ryangrim Then why did Democrats vote "no" on this bill instead of "present"?\nhttps://t.co/rxaLZAsCwi|@LeslieWix @lauraapollo @TIME So a few things https://t.co/rsHWVMMQSd\n\nYou also need to read the reason why are the Dems blocked the bill is not for the reason you think https://t.co/OBQoYVPnNp\n\nI am well awa

In [98]:
zs_data['primera_preds'].iloc[139]

"The delegates to the constitutional assembly had the conceit that they represented the people.But the people said something different” suggesting “there are limits to the next constitution. That’s a positive for Chile.” https://t.co/kZBdv2tB10|Rejection of Chile's overwrought draft constitution will stoke frustration &amp; anger but also gives the country a chance to reboot through that lost Latin American art - conciliation and consensus. w/ comments @NicSaldias, @juannagel\n"